# Needed modules

In [27]:
# Load needed libraries
import pandas as pd

In [28]:
# https://kioku-space.com/en/jupyter-skip-execution/
from IPython.core.magic import register_cell_magic # type: ignore

@register_cell_magic
def skip(line, cell):
    return

# Pickle save

In [29]:
%%skip
# =============================================================================
# Save the variables
# =============================================================================
variables_dict = {
}

# =============================================================================
# main function
# =============================================================================
def data_save_load(option, dict_variables=None):
    """
    This function is used to save or load data for the jupyter notebook
    """
    path_folder = "ipynb_db"  # Folder to save variables
    os.makedirs(path_folder, exist_ok=True)  # Create folder if not exist
    path_file = os.path.join(path_folder, "variables.pkl") # Path to save the variables

    if option == "save":
        with open(path_file, "wb") as f:
            pickle.dump(dict_variables, f)
    elif option == "load":
        with open(path_file, "rb") as f:
            variables = pickle.load(f)
        # Now load the variables
        for key, value in variables.items():
            variables[key] = value

# =============================================================================
# Call the function
# =============================================================================
data_save_load(option="save",
               dict_variables=variables_dict)

# 1. Load data

In [30]:
# Load new negative data:
neg_data = pd.read_csv("./2.Hallmarks_vs_NegData/negative_data_after_recaught.csv", sep=",", header=0)
print(neg_data.shape)
print(neg_data.dtypes)
neg_data.head()

(898, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LinJ.01,36104,36243,plus,GACAGACCGACACACGCAGCCGTGTGATGCCGCCGCCGAGGGCAGT...
1,LinJ.01,113760,114388,plus,CAGCGCCATGCACGACATGGCCGCTGACGTCCGTAGCCCTAACTCG...
2,LinJ.01,146413,146531,plus,CGAATTGTGTTCTGCGCATGCCTCTTCTCTGCCGTGCAGCATGCGG...
3,LinJ.01,261866,262439,plus,CGGACTTGGCAAGTGGCCGCCATCGATGAAAACGCACCATGCCTTT...
4,LinJ.01,271364,271651,plus,GAACGCCGCCCTCAATCGCGCGCTGAACTTCACGCGGCGGTCGACC...


In [31]:
# Load neg recaught data, i.e., data to add to the positive elements
neg_recaught_data = pd.read_csv("./2.Hallmarks_vs_NegData/neg_data_recaught_hits.csv", sep=",", header=0)
print(neg_recaught_data.shape)
print(neg_recaught_data.dtypes)
neg_recaught_data.head()

(16, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LinJ.15,81829,82005,plus,ATGGACTTCGAAAGTGGACGTGTTTCTCACAAATATGTCAATGCGC...
1,LinJ.15,87198,87893,plus,TTTATTTCAATGTGTCATTCTACCGCGAACACGTCAGCAAGTATAC...
2,LinJ.15,151897,152896,plus,TCTCGCTCGCCCCCGTCTTTCCTCTGTCTTTGCACTGTCGCATGCA...
3,LinJ.17,64185,64320,plus,CGCCGACGCCAACAGTCCACAGCTGCACCCACGGCCAGGCTGCAGG...
4,LinJ.17,64420,65085,plus,CCCTACCGCTGCCCACGAAACAGCAAGAACCGCCGTGCGCTCGCGG...


In [32]:
# Load positive data:
pos_data = pd.read_csv("./1.new_data/positive_database.csv", sep=",", header=0)
print(pos_data.shape)
print(pos_data.dtypes)
pos_data.head()

(2116, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LinJ.01,2,174,plus,CACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCAG...
1,LinJ.01,24094,24759,plus,GGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCATC...
2,LinJ.01,35372,35957,plus,CTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGCA...
3,LinJ.01,39791,40596,plus,TTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACAGC...
4,LinJ.01,54984,55548,plus,CTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGCA...


# 2. Add recaught data to the positive data

In [33]:
# Add neg_recaught_data to pos_data
pos_data = pd.concat([pos_data, neg_recaught_data], axis=0)

# Sort by ssqid, sstart
pos_data.sort_values(by=["sseqid", "sstart"], inplace=True)

# Check data
print(pos_data.shape)
print(pos_data.dtypes)
pos_data.head()

(2132, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LinJ.01,2,174,plus,CACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCAG...
1,LinJ.01,24094,24759,plus,GGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCATC...
2,LinJ.01,35372,35957,plus,CTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGCA...
3,LinJ.01,39791,40596,plus,TTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACAGC...
4,LinJ.01,54984,55548,plus,CTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGCA...


# 3. Check if the negative and positive data for included or overlaps

In [34]:
# Copy og data
pos_data_check = pos_data.copy()
neg_data_check = neg_data.copy()

In [35]:
# Create interval columns
pos_data_check["interval"] = pos_data_check.apply(lambda x: pd.Interval(left=x["sstart"], right=x["send"], closed="both"), axis=1)
neg_data_check["interval"] = neg_data_check.apply(lambda x: pd.Interval(left=x["sstart"], right=x["send"], closed="both"), axis=1)

print(pos_data_check.loc[:, ["sseqid", "sstart", "send", "interval"]].head())
print(neg_data_check.loc[:, ["sseqid", "sstart", "send", "interval"]].head())

    sseqid  sstart   send        interval
0  LinJ.01       2    174        [2, 174]
1  LinJ.01   24094  24759  [24094, 24759]
2  LinJ.01   35372  35957  [35372, 35957]
3  LinJ.01   39791  40596  [39791, 40596]
4  LinJ.01   54984  55548  [54984, 55548]
    sseqid  sstart    send          interval
0  LinJ.01   36104   36243    [36104, 36243]
1  LinJ.01  113760  114388  [113760, 114388]
2  LinJ.01  146413  146531  [146413, 146531]
3  LinJ.01  261866  262439  [261866, 262439]
4  LinJ.01  271364  271651  [271364, 271651]


In [36]:
# Group the elements
pos_data_grouped = pos_data_check.groupby("sseqid")
neg_data_grouped = neg_data_check.groupby("sseqid")

In [39]:
# Let's check for each group of positive elements if there is any negative if the elements are exactly the same, overlap or are inside one another
same_elems = []
overlaping_elems = []
inside_elems = []
inside_elems2 = []
for name1, data1 in pos_data_grouped:
    for name2, data2 in neg_data_grouped:
        if name2 != name1:
            # print(f">> SKIPPING Chrom. Chromosome in negative elements {name2} is different from the one in positive elements {name1}")
            continue
        else:
            print("="*50)
            print(f"Checking chromosome {name1}")
            # Checking for same coordinates
            for _, elem1 in data1.iterrows():
                for _, elem2 in data2.iterrows():
                    if elem1["interval"] == elem2["interval"]:
                        print(f"Found same element in chromosome {name1}")
                        same_elems.append(elem1)
                    elif elem1["interval"].overlaps(elem2["interval"]):
                        print(f"Found overlaping element in chromosome {name1}")
                        overlaping_elems.append([elem1, elem2])
                    else:
                        if (elem1["sstart"] > elem2["sstart"] and elem1["sstart"] < elem2["send"] /
                            elem1["send"] < elem2["send"] and elem1["send"] > elem2["sstart"]):
                            print(f"Found element inside another element in chromosome {name1}")
                            inside_elems.append(elem1)
                        if (elem2["sstart"] > elem1["sstart"] and elem2["sstart"] < elem1["send"] /
                            elem2["send"] < elem1["send"] and elem2["send"] > elem1["sstart"]):
                            print(f"Found element inside another element in chromosome {name1}")
                            inside_elems2.append(elem1)

print("*"*50)
print(f"Found {len(same_elems)} elements that are the same")
print(f"Found {len(overlaping_elems)} elements that are overlaping")
print(f"Found {len(inside_elems)} elements that are inside another element")
print(f"Found {len(inside_elems2)} elements that are inside another element")


Checking chromosome LinJ.01
Checking chromosome LinJ.02
Checking chromosome LinJ.03
Checking chromosome LinJ.04
Checking chromosome LinJ.05
Checking chromosome LinJ.06
Checking chromosome LinJ.07
Checking chromosome LinJ.08
Checking chromosome LinJ.09
Checking chromosome LinJ.10
Checking chromosome LinJ.11
Checking chromosome LinJ.12
Checking chromosome LinJ.13
Checking chromosome LinJ.14
Checking chromosome LinJ.15
Checking chromosome LinJ.16
Checking chromosome LinJ.17
Checking chromosome LinJ.18
Checking chromosome LinJ.19
Checking chromosome LinJ.20
Checking chromosome LinJ.21
Checking chromosome LinJ.22
Checking chromosome LinJ.23
Checking chromosome LinJ.24
Checking chromosome LinJ.25
Checking chromosome LinJ.26
Checking chromosome LinJ.27
Checking chromosome LinJ.28
Checking chromosome LinJ.29
Checking chromosome LinJ.30
Checking chromosome LinJ.31
Checking chromosome LinJ.32
Checking chromosome LinJ.33
Checking chromosome LinJ.34
Checking chromosome LinJ.35
Found overlaping ele

In [51]:
for x in overlaping_elems:
    print("pos data:")
    print(x[0][["sseqid", "sstart", "send", "interval"]])
    print(x[0]["sseq"])
    print("="*50)
    print("neg data:")
    print(x[1][["sseqid", "sstart", "send", "interval"]])
    print(x[1]["sseq"])


pos data:
sseqid                 LinJ.35
sstart                 1630851
send                   1631161
interval    [1630851, 1631161]
Name: 1922, dtype: object
CACTTCAGCGTGGCATCACAAGGTCCGGTGCACCAGCCTTGCGTGGGATAGCCGAGCAGCCCACCCTATCCCTGCTAGTCCCGGACCGCTTCTGCTGGTGACAGGGCCACGCGCCCACGACGTGAGGGAGGGGAAGCCAGTGCGATTTTGTCGCTACCGATGTCGGCGGTGAGGCCCTGGACGGCGTTGTGCTGGGGCAGCCTGCGACTGCGCACACGTCTGCGCCATCCATATGATTAGGCAGCGTGTCAGCGTGGCTGGAGCGCATCTCTCGCCCGGCCCTCACACTGCCTTGCTGGTGCGGGGATGCA
neg data:
sseqid                 LinJ.35
sstart                 1631161
send                   1631853
interval    [1631161, 1631853]
Name: 837, dtype: object
ACCGGGTGGCGACCGGCAAAGTTGGAGCGGCCGTGAGGCGACCTGCGGAGCGGGGGATGAGTGGGTGCACAGCATGAGAGAGAGGCCGCGCGCGGATCACTGTCTTGGGGCATTGCGGTAATGCGCGTGGGCAGTGCTGCGTCGCACCACGCGACGGGACACGTAACAGGGTCACGGGGAGGGTAGGGTGGCGCTTGAACTCATGCTCTATGGCAGAGAATGGACACACGGCACAAGGAATAGTCTGTTTCTCTCTGTGCCCGTTACTTTCCTCCTTTTTTTTACTGTGTGAGTTTTTTTTCTATATGGTCTTTTTTGTTTGCCTTCATCTCATCATCGTTAGTGCGTGAGGTGATGCACCTTTTCGGT

In [52]:
# Load test data from the beginning
test_pos = pd.read_csv("./1.new_data/positive_database.csv", sep=",", header=0)
test_neg = pd.read_csv("./1.new_data/negative_database.csv", sep=",", header=0)

In [53]:
test_pos[
    test_pos.loc[:, "sseq"] == "CACTTCAGCGTGGCATCACAAGGTCCGGTGCACCAGCCTTGCGTGGGATAGCCGAGCAGCCCACCCTATCCCTGCTAGTCCCGGACCGCTTCTGCTGGTGACAGGGCCACGCGCCCACGACGTGAGGGAGGGGAAGCCAGTGCGATTTTGTCGCTACCGATGTCGGCGGTGAGGCCCTGGACGGCGTTGTGCTGGGGCAGCCTGCGACTGCGCACACGTCTGCGCCATCCATATGATTAGGCAGCGTGTCAGCGTGGCTGGAGCGCATCTCTCGCCCGGCCCTCACACTGCCTTGCTGGTGCGGGGATGCA"
    ]

,sseqid,sstart,send,sstrand,sseq
1922,LinJ.35,1630851,1631161,plus,CACTTCAGCGTGGCATCACAAGGTCCGGTGCACCAGCCTTGCGTGG...


In [54]:
test_neg[
    test_neg.loc[:, "sseq"] == "ACCGGGTGGCGACCGGCAAAGTTGGAGCGGCCGTGAGGCGACCTGCGGAGCGGGGGATGAGTGGGTGCACAGCATGAGAGAGAGGCCGCGCGCGGATCACTGTCTTGGGGCATTGCGGTAATGCGCGTGGGCAGTGCTGCGTCGCACCACGCGACGGGACACGTAACAGGGTCACGGGGAGGGTAGGGTGGCGCTTGAACTCATGCTCTATGGCAGAGAATGGACACACGGCACAAGGAATAGTCTGTTTCTCTCTGTGCCCGTTACTTTCCTCCTTTTTTTTACTGTGTGAGTTTTTTTTCTATATGGTCTTTTTTGTTTGCCTTCATCTCATCATCGTTAGTGCGTGAGGTGATGCACCTTTTCGGTGACGCTCCTCTATTTTGTTTTTCAGGTTTCGTTCGGTTTTCGTCATTCGTGTATACCATGATCTTCGCGTGGTAAGCTTAATATTCTGTAGATGGATTTCCACATGACGGCAGGCATCCTGATCGGAGAGGCTCTTCTTTCTGTTACTGCCTGCTGCTGTTTACAGTATTATATGAATACTCTTTTTCTGTTTAGATTTTGCATCCTCCTTGTTCTTCGTTTTTCTTTCTTTTCAAATCTTCTGCCGCTGCTTAGCGCATCTCCGCGTGGACCCCATTCGTTCCCTTTTCCTGTGGTGAACTGTTTCTTGCTTGACCCTTTCCC"
    ]

,sseqid,sstart,send,sstrand,sseq
852,LinJ.35,1631161,1631853,plus,ACCGGGTGGCGACCGGCAAAGTTGGAGCGGCCGTGAGGCGACCTGC...


In [60]:
test_pos2 = pd.read_csv("../17.Real_coordinates_method/data/consensus+LmSIDER2A+B/positive_data+recaught.csv", sep=",", header=0)
test_neg2 = pd.read_csv("../17.Real_coordinates_method/data/consensus+LmSIDER2A+B/neg_data_corrected.csv", sep=",", header=0)
print(test_pos2.shape)
print(test_neg2.shape)

(2092, 5)
(691, 5)


In [62]:
test_pos2[
    (test_pos2["sseqid"] == "LinJ.35") & (test_pos2["sstart"] == 1630851)
    ]

,sseqid,sstart,send,sstrand,sseq


In [63]:
test_neg2[
    (test_neg2["sseqid"] == "LinJ.35") & (test_neg2["sstart"] == 1630851)
]

,sseqid,sstart,send,sstrand,sseq
